In [8]:
import json
import random
import os

In [9]:
# 读取 JSON 文件
with open("/Users/liuyiwei/Desktop/TactfulToM/dataset/Tactful_conv_element.json", "r", encoding="utf-8") as file:
    data = json.load(file)

In [10]:
first_item = data[0]
first_item

{'set_id': '1-0-0',
 'lie_id': '0-0',
 'conv_id': 1,
 'truth_id': 0,
 'lie_type': 'altruistic_white_lies',
 'emotion': 'sad',
 'topic': {'scenario': 'favorite cuisine styles',
  'situation_topic': 'the plan for this weekend',
  'situation': 'Pearl is inviting Jamey for having dinner at a new opened fancy Italian restaurant together this weekend',
  'lie_objective': 'decline the invitation',
  'leave_reason_B': 'need to make a phone call',
  'leave_reason_D_1': 'remembered to submit some papers',
  'leave_reason_D_2': 'have a meeting starting soon that I need to prepare for'},
 'relationship': 'coworkers',
 'characters': {'liar': 'Jamey',
  'target': 'Pearl',
  'accomplice': 'Marithza',
  'observer': 'Donyae'},
 'lie': {'real_reason_c': 'Jamey does not want to dine out at expensive restaurants, preferring more casual dining options instead.',
  'lie_c': 'Jamey need to work this weekend, so Jamey can’t make it.',
  'truth_c': '',
  'falsification': None},
 'full_context': "**Pearl**: So,

In [11]:
# gpt rewrite 一下real_reason_c
real_reason_q = "Alice’s social battery is really running dry these days, and she thinks she needs more time to be alone."
lie_q = "Alice has a meeting in the afternoon, so she won’t be able to join for studying."
truth_q = "Alice is actually free for the whole afternoon."

### 1. comprehensionQA & justificationQA

In [39]:
def generate_comprehensionQA(data):
    liar = data["characters"]["liar"]
    target = data["characters"]["target"]
    lie = data["lie"]["lie_c"].rstrip(".")
    
    comprehension_qa = {
        "question": f"Was what {liar} told {target}, '{lie}', true?",
        "question_type": "comprehension",
        "correct_answer": "No",
        "wrong_answers": "Yes"
    }
    
    return {"comprehensionQA": [comprehension_qa]}  # **返回 JSON 结构**

In [40]:
print(json.dumps(generate_comprehensionQA(first_item), indent=4))

{
    "comprehensionQA": [
        {
            "question": "Was what Jamey told Pearl, 'Jamey need to work this weekend, so Jamey can\u2019t make it', true?",
            "question_type": "comprehension",
            "correct_answer": "No",
            "wrong_answer": "Yes"
        }
    ]
}


In [17]:
def load_justification_options(file_path):
    """
    从指定的 JSON 文件中加载各 set_id 对应的 justification 选项集
    """
    with open(file_path, "r", encoding="utf-8") as f:
        options_by_set = json.load(f)
    return options_by_set

In [41]:
# freefrom // mcq
def generate_justificationQA(data):
    set_id = data["set_id"]

    options_by_set = load_justification_options("/Users/liuyiwei/Desktop/TactfulToM/dataset/justification_option.json")
    options = options_by_set.get(set_id)
    if options is None:
        raise ValueError(f"Options not found: {set_id}")
    
    liar = data["characters"]["liar"]
    target = data["characters"]["target"]
    lie = data["lie"]["lie_c"].rstrip(".")
    real_reason = data["lie"]["real_reason_c"].rstrip(".")

    justification_qas = [
        {
            "question": f"Why did {liar} tell {target} '{lie}'?",
            "question_type": "justification:truth",
            "correct_answer":options["truth"]["correct_answer"],
            "wrong_answers": options["truth"]["wrong_answers"]
        },
        # 这个问题的核心变成强调了 lie vs real reason 的对比，突出强调了说谎者的动机中存在一个真实的但不便明说的原因（即真实情绪或心理原因）。因此，在设计选项时，需要稍作调整，突出 避免负面影响、礼貌、维护关系 等。
        {   
            "question": f"Why did {liar} tell {target} the '{lie}' instead of the '{real_reason}'?",
            "question_type": "justification:real_reason",
            "correct_answer":options["real_reason"]["correct_answer"],
            "wrong_answers": options["real_reason"]["wrong_answers"]
        }
    ]

    return {"justificationQA": justification_qas}  # 返回 JSON 结构

In [44]:
# print(json.dumps(generate_justificationQA(first_item), indent=4))

### 2. fact_QA

In [21]:
# mcq
# dist
def generate_fact_QA(data):
    liar = data["characters"]["liar"]
    target = data["characters"]["target"]
    accomplice = data["characters"]["accomplice"]
    observer = data["characters"]["observer"]

    real_reason = data["lie"]["real_reason_c"].rstrip(".")
    truth =  data["lie"]["truth_c"]
    truth_id = data["truth_id"]

    # Because the correct answer has liar, make the incorrect choices all have liar and acomplice as well
    # anyway, is to make the incorrect choices a little more confusing
    # f"{observer}, {accomplice} and {target} talked about that.",
    # f"{observer} and {target} talked about that",
    # f"{observer}, {target} and {liar} talked about that.",
    wrong_answers_reason = [
        f"{accomplice} and {target} talked about that.",
        f"{observer} and {liar} talked about that.",
        f"{observer} and {accomplice} talked about that.",
        f"{target} and {liar} talked about that.",
        f"{target}, {accomplice} and {liar} talked about that.",
        f"{liar}, {observer} and {accomplice} talked about that."
    ]

    random_wrong_answers_reason = random.sample(wrong_answers_reason, 3)

    # **1️⃣ ReasonQA**
    reason_qa = {
        "question": f"Who discussed '{real_reason}'?",
        "question_type": "fact:real_reason",
        "correct_answer": f"{liar} and {accomplice} talked about that.",
        "wrong_answers": random_wrong_answers_reason 
    }

    # **2️⃣ truthQA（only when truth_id == 1）**
    truth_qa = None
    if truth_id == 1:
        truth =  truth.rstrip(".")

        random_wrong_answers_truth = random.sample(wrong_answers_truth, 3)
        # f"{target} and {liar} talked about that"
        # f"{observer} and {target} talked about that",
        # f"{accomplice} and {target} talked about that",
        wrong_answers_truth = [
            f"{observer}, {accomplice} and {target} talked about that.",
            f"{observer}, {target} and {liar} talked about that.",
            f"{target}, {accomplice} and {liar} talked about that."
            f"{liar} and {accomplice} talked about that",
            f"{observer} and {liar} talked about that",
            f"{observer} and {accomplice} talked about that",
        ]

        truth_qa = {
            "question": f"Who discussed '{truth}'?",
            "question_type": "fact:truth",
            "correct_answer": f"{liar}, {observer} and {accomplice} talked about that.",
            "wrong_answers": random_wrong_answers_truth
            }

    return {"fact_reasonQA": [reason_qa], "fact_truthQA": [truth_qa] if truth_qa else []}

In [99]:
print(json.dumps(generate_fact_QA(data[0]), indent=4))

{
    "fact_reasonQA": [
        {
            "question": "Who discussed 'Jamey does not want to dine out at expensive restaurants, preferring more casual dining options instead'?",
            "question_type": "fact:real_reason",
            "correct_answer": "Jamey and Marithza talked about that.",
            "wrong_answer": [
                "Jamey, Donyae and Marithza talked about that.",
                "Donyae and Jamey talked about that.",
                "Marithza and Pearl talked about that."
            ]
        }
    ],
    "fact_truthQA": []
}


### 3. beliefQAs

In [ ]:
# The text provides no information on 'What does {liar} believe about '{real_reason_question}'. Therefore, it can be stated that {liar} is unaware about this information.

# {liar} is unaware of who discussed it.

# Jaiden believes that Veronica is unaware of the strategies Christian uses to manage stress in a high-pressure work environment, because Veronica was not involved in the conversation when this was discussed.


In [ ]:
# 一阶的
# A is believs that 
#   can be the true 
#   or wrong one

# A is unaware of xx
#   The text provide no information on xx, so it can be stated that {liar} is unaware about xx.
#   Based on the text, xxx

In [ ]:
# 二阶的
# A believs B believes that (can be the true one or wrong one)
# A believs B is unaware of xxx
# A is unaware of B believs that (can be the true one or wrong one)
# A is unaware of what believs of.
# the text provide no information on xx, it can be stated that {liar} is unaware about this information.

In [22]:
def add_prefix(option, prefix="The text did not provide enough information about this, so we can state that "):
    """
    随机决定是否在传入的字符串前加上前缀。

    参数：
    - option: 原始选项字符串。
    - prefix: 前缀字符串，默认为 "The text did not provide enough information about this, so we can state that "。

    返回：
    - 根据随机结果返回原始选项或加上前缀后的选项。
    """
    # if random.choice([True, False]):
    option = prefix + option
    return option

In [23]:
def generate_1stbeliefQAs(data):
    liar = data["characters"]["liar"]
    target = data["characters"]["target"]
    accomplice = data["characters"]["accomplice"]
    observer = data["characters"]["observer"]
    
    real_reason = data["lie"]["real_reason_c"].rstrip(".")
    # truth =  data["lie"]["truth_c"].rstrip(".")
    truth = (data["lie"].get("truth_c") or "").rstrip(".")
    truth_id = data["truth_id"]
    truth_question = f"Who discussed '{truth}'?" if truth_id == 1 else None
    real_reason_question = f"Who discussed '{real_reason}'?"
    

    belief_qas = []

    #######################################
    # **first-order belief（real reason）**
    # {liar}
    # +-
    first_wrong_options_liar = [
        f"{liar} believes that {target} discussed {real_reason}.",
        f"{liar} believes that {observer} discussed {real_reason}.",
        f"{liar} believes that {observer} and {target} discussed {real_reason}.",
        f"{liar} believes that {accomplice} and {target} discussed {real_reason}.",
        f"{liar} believes that {observer} and {accomplice} discussed {real_reason}.",
    ]
    # -+
    second_wrong_option_liar = f"{liar} is unaware of {accomplice} discussed {real_reason}."
    # --
    third_wrong_options_liar = [
        f"{liar} is unaware of {target} discussed {real_reason}.",
        f"{liar} is unaware of {observer} discussed {real_reason}.",
        f"{liar} is unaware of {target} and {observer} discussed {real_reason}.",
        f"{liar} is unaware of {accomplice} and {observer} discussed {real_reason}.",
        f"{liar} is unaware of {target} and {accomplice} discussed {real_reason}.",
    ]
    belief_qas.append({
        "question": f"What does {liar} believe about '{real_reason_question}'?",
        "question_type": "tom:belief:accessible:reason",
        "tom_type": "first-order",
        "correct_answer": f"{liar} believes that {accomplice} discussed {real_reason}.", # ++
        "wrong_answers": [
                # +-
                random.choice(first_wrong_options_liar),
                # -+
                add_prefix(second_wrong_option_liar),
                # --
                add_prefix(random.choice( third_wrong_options_liar))
                ],
        "real_reason_accessibility": "accessible"
    })

    # {target}
    # +-
    first_wrong_options_target = [
        f"{target} is unaware of {observer} discussed {real_reason}.",
        f"{target} is unaware of {liar} and {observer} discussed {real_reason}.",
        f"{target} is unaware of {accomplice} and {observer} discussed {real_reason}."
    ]
    # -+
    second_wrong_option_target = [
        f"{target} believes that {liar} and {accomplice} discussed {real_reason}.",
        f"{target} believes that {liar} discussed {real_reason}.",
        f"{target} believes that {accomplice} discussed {real_reason}."
    ]
    # --
    third_wrong_options_target = [
        f"{target} believes that {accomplice} and {observer} discussed {real_reason}.",
        f"{target} believes that {liar}  and {observer} discussed {real_reason}.",
        f"{target} believes that {observer} discussed {real_reason}."
    ]
    correct_answer = f"{target} is unaware of {liar} and {accomplice} discussed {real_reason}."

    belief_qas.append({
        "question": f"What does {target} believe about '{real_reason_question}'?",
        "question_type": "tom:belief:inaccessible:reason",
        "tom_type": "first-order",
        "correct_answer": add_prefix(correct_answer),
        "wrong_answers": [
            # +-
            add_prefix(random.choice(first_wrong_options_target)),
            # -+
            random.choice(second_wrong_option_target),
            # --
            random.choice(third_wrong_options_target)
        ],
        "real_reason_accessibility": "inaccessible"
    })

    # {accomplice}
    # +-
    first_wrong_options_accomplice = [
        f"{accomplice} believes that {observer} discussed {real_reason}.",
        f"{accomplice} believes that {target} discussed {real_reason}.",
        f"{accomplice} believes that {observer} and {target} discussed {real_reason}.",
        f"{accomplice} believes that {observer} and {liar} discussed {real_reason}.",
        f"{accomplice} believes that {liar} and {target} discussed {real_reason}."
    ]
    # -+
    second_wrong_option_accomplice = f"{accomplice} is unaware of {liar} discussed {real_reason}."
    # --
    third_wrong_options_accomplice = [
        f"{accomplice} is unaware of {observer} discussed {real_reason}.",
        f"{accomplice} is unaware of {target} discussed {real_reason}.",
        f"{accomplice} is unaware of {observer} and {target} discussed {real_reason}.",
        f"{accomplice} is unaware of {observer} and {liar} discussed {real_reason}.",
        f"{accomplice} is unaware of {liar} and {target} discussed {real_reason}."
    ]
    belief_qas.append({
        "question": f"What does {accomplice} believe about '{real_reason_question}'?",
        "question_type": "tom:belief:accessible:reason",
        "tom_type": "first-order",
        "correct_answer": f"{accomplice} believes that {liar} discussed {real_reason}.", # ++
        "wrong_answers": [
                # +-
                random.choice(first_wrong_options_accomplice),
                # -+
                add_prefix(second_wrong_option_accomplice),
                # --
                add_prefix(random.choice(third_wrong_options_accomplice))
                ],
        "real_reason_accessibility": "accessible"
    })
    #######################################

    #######################################
    # **第一阶信念（关于 truth_question）**
    # **如果 truthQA 不是空的，加入关于 truth_question 的 belief question**
    if truth_id == 1 and truth_question:
        # {liar}
        # +-
        first_wrong_options_liar_truth = [
            f"{liar} believes that {target} discussed {truth}.",
            f"{liar} believes that {observer} and {target} discussed {truth}.",
            f"{liar} believes that {accomplice} and {target} discussed {truth}.",
        ]
        # -+
        second_wrong_option_liar_truth = [
            f"{liar} is unaware of {accomplice} and {observer} discussed {truth}.",
            f"{liar} is unaware of {accomplice} discussed {truth}.",
            f"{liar} is unaware of {observer} discussed {truth}."
        ]
        # --
        third_wrong_options_liar_truth = [
            f"{liar} is unaware of {target} discussed {truth}.",
            f"{liar} is unaware of {observer} and {target} discussed {truth}.",
            f"{liar} is unaware of {accomplice} and {target} discussed {truth}.",
        ]
        belief_qas.append({
        "question": f"What does {liar} believe about '{truth_question}'?",
        "question_type": "tom:belief:accessible:truth",
        "tom_type": "first-order",
        "correct_answer": f"{liar} believes that {accomplice} and {observer} discussed {truth}.",
        "wrong_answers": [
                # +-
                random.choice(first_wrong_options_accomplice),
                # -+
                add_prefix(random.choice(second_wrong_option_liar_truth)),
                # --
                add_prefix(random.choice(third_wrong_options_liar_truth))
                ],
        "truth_accessibility": "accessible"
    })
        
        # {target}
        # +-
        # first_wrong_options_target_truth = [
        #     f"{target} is unaware of {observer} discussed {real_reason}.",
        #     f"{target} is unaware of {liar} and {observer} discussed {real_reason}.",
        #     f"{target} is unaware of {accomplice} and {observer} discussed {real_reason}."
        # ]
        # -+
        second_wrong_option_target_truth = [
            f"{target} believes that {liar}, {accomplice} and {observer} discussed about {truth}.",
            f"{target} believes that {accomplice} and {observer} discussed about {truth}.",
            f"{target} believes that {liar} and {observer} discussed about {truth}.",
            f"{target} believes that {liar} and {accomplice} discussed about {truth}.",
            f"{target} believes that {liar} discussed about {truth}.",
            f"{target} believes that {accomplice} discussed about {truth}.",
            f"{target} believes that {observer} discussed about {truth}.",
        ]
        # --
        # third_wrong_options_target_truth = [
        #     f"{target} believes that {accomplice} and {observer} discussed {real_reason}.",
        #     f"{target} believes that {liar}  and {observer} discussed {real_reason}.",
        #     f"{target} believes that {observer} discussed {real_reason}."
        # ]
        # ++
        correct_answer = f"{target} is unaware of {liar}, {accomplice} and {observer} discussed about {truth}."
        belief_qas.append({
        "question": f"What does {target} believe about '{truth_question}'?",
        "question_type": "tom:belief:inaccessible:truth",
        "tom_type": "first-order",
        "correct_answer": add_prefix(correct_answer),
        "wrong_answers": [
            # # +-
            # add_prefix(random.choice(first_wrong_options_target)),
            # -+
            random.sample(second_wrong_option_target_truth, 3),
            # # --
            # random.choice(third_wrong_options_target)
        ],
        "truth_accessibility": "inaccessible"
    })
        
        # {accomplice}
        # +-
        first_wrong_options_accomplice_truth = [
            f"{accomplice} believes that {target} discussed {truth}.",
            f"{accomplice} believes that {liar} and {target} discussed {truth}.",
            f"{accomplice} believes that {observer} and {target} discussed {truth}."
        ]
        # -+
        second_wrong_option_accomplice_truth = [
            f"{accomplice} is unaware of {liar} discussed {truth}.",
            f"{accomplice} is unaware of {observer} discussed {truth}.",
            f"{accomplice} is unaware of {liar} and {observer} discussed {truth}.",
        ]
        # --
        third_wrong_options_accomplice_truth = [
            f"{accomplice} is unaware of {target} discussed {truth}.",
            f"{accomplice} is unaware of {liar} and {target} discussed {truth}.",
            f"{accomplice} is unaware of {observer} and {target} discussed {truth}."
        ]

        belief_qas.append({
        "question": f"What does {target} believe about '{truth_question}'?",
        "question_type": "tom:belief:accessible:truth",
        "tom_type": "first-order",
        "correct_answer": f"{accomplice} believe that {liar} and {observer} discussed {truth}.", # ++
        "wrong_answers": [
                # +-
                random.choice(first_wrong_options_accomplice_truth),
                # -+
                add_prefix(random.choice(second_wrong_option_accomplice_truth)),
                # --
                add_prefix(random.choice(third_wrong_options_accomplice_truth))
                ],
        "truth_accessibility": "accessible"
    })

    return {"beliefQAs": belief_qas}

In [80]:
def generate_2ndbeliefQAs(data):
    liar = data["characters"]["liar"]
    target = data["characters"]["target"]
    accomplice = data["characters"]["accomplice"]
    observer = data["characters"]["observer"]


    real_reason = data["lie"]["real_reason_c"].rstrip(".")
    # truth = data["lie"]["truth_c"].rstrip(".")
    truth = (data["lie"].get("truth_c") or "").rstrip(".")
    truth_id = data["truth_id"]
    truth_question = f"Who discussed '{truth}'?" if truth_id == 1 else None
    real_reason_question = f"Who discussed '{real_reason}'?"
    

    belief_qas = []
    
    #######################################
    # **第二阶信念（关于 real reason）**
    ########### {liar} ########
    # 还有长答案和短答案
    # Jaiden believes that Veronica is unaware of that.
    
    # A不在对话里面 --inaccessible
    # The text provides no information on what Jaiden believes about Veronica's belief regarding xx, Therefore, it can be stated that Jaiden is unaware about this information.
    # A在对话里面，B不在对话里面 --accessible
    # A believes that B is unaware of the strategies Christian uses to manage stress in a high-pressure work environment, because B was not involved in the conversation when this was discussed.",
    
    # AB
    # liar:target
    belief_qas.append({
        "question": f"What does {liar} believe about {target}'s belief regarding '{real_reason_question}'?",
        "question_type": "tom:belief:accessible:reason",
        "tom_type": "second-order",
        # ++
        "correct_answer": f"{liar} believes that {target} is unaware of that {real_reason}, because {target} was not involved in the conversation when this was discussed.",
        "wrong_answers": [
            # +-
            f"{liar} believes that {target} believs that {real_reason}.",
            # -+
            f"The text provides no information on what {liar} believes about {target}'s belief regarding that. Therefore, it can be stated that {liar} is unaware of {target} is unaware of {real_reason}.",
            # --
            f"The text provides no information on what {liar} believes about {target}'s belief regarding that. Therefore, it can be stated that {liar} is unaware of {target} believs that {real_reason}."
        ],
        "real_reason_accessibility": "accessible"
    })
    # AC
    # liar:accomplice
    # Who discussed 'real_reason'?"
    belief_qas.append({
        "question": f"What does {liar} believe about {accomplice}'s belief regarding '{real_reason_question}'?",
        "question_type": "tom:belief:accessible:reason",
        "tom_type": "second-order",
        # ++
        "correct_answer": f"{liar} believes that {accomplice} believes that {real_reason}.",
        "wrong_answers": [
            # +-
            f"{liar} believes that {accomplice} is unaware of {real_reason}, because {accomplice} was not involved in the conversation when this was discussed.",
            # -+
            f"The text provides no information on what {liar} believes about {accomplice}'s belief regarding that. Therefore, it can be stated that {liar} is unaware of that {accomplice} is unaware of {real_reason}.",
            # --
            f"The text provides no information on what {liar} believes about {accomplice}'s belief regarding that. Therefore, it can be stated that {liar} is unaware of that {accomplice} believs that {real_reason}."
        ],
        "real_reason_accessibility": "accessible"
    })
    # AD
    # liar:observer
    belief_qas.append({
        "question": f"What does {liar} believe about {observer}'s belief regarding '{real_reason_question}'?",
        "question_type": "tom:belief:accessible:reason",
        "tom_type": "second-order",
        # ++
        "correct_answer":f"{liar} believes that {observer} is unaware of that {real_reason}, because {observer} was not involved in the conversation when this was discussed.",
        "wrong_answers": [
            # +-
            f"{liar} believes that {observer} believs that {real_reason}.",
            # -+
            f"The text provides no information on what {liar} believes about {observer}'s belief regarding that. Therefore, it can be stated that {liar} is unaware of that {target} is unaware of {real_reason}.",
            # --
            f"The text provides no information on what {liar} believes about {observer}'s belief regarding that. Therefore, it can be stated that {liar} is unaware of that {target} believs that {real_reason}."
        ],
        "real_reason_accessibility": "accessible"
    })

    ########### {target} ###########
    # BA
    # target:liar
    belief_qas.append({
        "question": f"What does {target} believe about {liar}'s belief regarding '{real_reason_question}'?",
        "question_type": "tom:belief:inaccessible:reason",
        "tom_type": "second-order",
        # ++
        "correct_answer": f"The text provides no information on what {target} believes about {liar}'s belief regarding that. Therefore, it can be stated that {target} is unaware of that {liar} believes that {real_reason}.",
        "wrong_answers": [
            # +-
            f"The text provides no information on what {target} believes about {liar}'s belief regarding that. Therefore, it can be stated that {target} is unaware of that {liar} is unaware of {real_reason}.",
            # -+
            f"{target} believes that {liar} believs that {real_reason}.",
            # --
            f"{target} believes that {liar} is unaware of that {real_reason}, because {liar} was not involved in the conversation when this was discussed."
        ],
        "real_reason_accessibility": "inaccessible"
    })
    # BC
    # target:accomplice
    belief_qas.append({
        "question": f"What does {target} believe about {accomplice}'s belief regarding '{real_reason_question}'?",
        "question_type": "tom:belief:inaccessible:reason",
        "tom_type": "second-order",
        # ++
        "correct_answer": f"The text provides no information on what {target} believes about {accomplice}'s belief regarding that. Therefore, it can be stated that {target} is unaware of that {accomplice} believes that {real_reason}.",
        "wrong_answers": [
            # +-
            f"The text provides no information on what {target} believes about {accomplice}'s belief regarding that. Therefore, it can be stated that {target} is unaware of that {accomplice} is unaware of {real_reason}.",
            # -+
            f"{target} believes that {accomplice} believs that {real_reason}.",
            # --
            f"{target} believes that {accomplice} is unaware of that {real_reason}, because {accomplice} was not involved in the conversation when this was discussed."
        ],
        "missed_info_accessibility": "inaccessible"
    })

    # BD
    # target:observer
    belief_qas.append({
        "question": f"What does {target} believe about {observer}'s belief regarding '{real_reason_question}'?",
        "question_type": "tom:belief:inaccessible:reason",
        "tom_type": "second-order",
        # ++
        "correct_answer": f"The text provides no information on what {target} believes about {observer}'s belief regarding that. Therefore, it can be stated that {target} is unaware of that {observer} is unaware of {real_reason}.",
        "wrong_answers": [
            # +-
            f"The text provides no information on what {target} believes about {observer}'s belief regarding that. Therefore, it can be stated that {target} is unaware of that {observer} believes that {real_reason}.",
            # -+
           f"{target} believes that {observer} is unaware of that {real_reason}, because {observer} was not involved in the conversation when this was discussed.",
            # --
            f"{target} believes that {observer} believs that {real_reason}."
        ],
        # "wrong_answers": f"{target} believes that {observer} is unaware of that.",
        "missed_info_accessibility": "inaccessible"
    })

    ########### {accomplice} ########
    # CA
    # accomplice:liar
    belief_qas.append({
        "question": f"What does {accomplice} believe about {liar}'s belief regarding '{real_reason_question}'?",
        "question_type": "tom:belief:accessible:reason",
        "tom_type": "second-order",
        # ++
        "correct_answer": f"{accomplice} believes that {liar} believes that {real_reason}.",
        "wrong_answers": [
            # +-
            f"{accomplice} believes that {liar} is unaware of {real_reason}, because {liar} was not involved in the conversation when this was discussed.",
            # -+
            f"The text provides no information on what {accomplice} believes about {liar}'s belief regarding that. Therefore, it can be stated that {accomplice} is unaware of that {liar} is unaware of {real_reason}.",
            # --
            f"The text provides no information on what {accomplice} believes about {liar}'s belief regarding that. Therefore, it can be stated that {accomplice} is unaware of that {liar} believs that {real_reason}."
        ],
        "real_reason_accessibility": "accessible"
    })
    # accomplice:target
    # CB
    belief_qas.append({
        "question": f"What does {accomplice} believe about {target}'s belief regarding '{real_reason_question}'?",
        "question_type": "tom:belief:accessible:reason",
        "tom_type": "second-order",
        # ++
        "correct_answer": f"{accomplice} believes that {target} is unaware of that {real_reason}, because {target} was not involved in the conversation when this was discussed.",
        "wrong_answers": [
            # +-
            f"{accomplice} believes that {target} believs that {real_reason}.",
            # -+
            f"The text provides no information on what {accomplice} believes about {target}'s belief regarding that. Therefore, it can be stated that {accomplice} is unaware of {target} is unaware of {real_reason}.",
            # --
            f"The text provides no information on what {accomplice} believes about {target}'s belief regarding that. Therefore, it can be stated that {accomplice} is unaware of {target} believs that {real_reason}."
        ],
        "real_reason_accessibility": "accessible"
    })

    # accomplice:observer
    # CD
    belief_qas.append({
        "question": f"What does {accomplice} believe about {observer}'s belief regarding '{real_reason_question}'?",
        "question_type": "tom:belief:accessible:reason",
        "tom_type": "second-order",
        # ++
        "correct_answer":f"{accomplice} believes that {observer} is unaware of that {real_reason}, because {observer} was not involved in the conversation when this was discussed.",
        "wrong_answers": [
            # +-
            f"{accomplice} believes that {observer} believs that {real_reason}.",
            # -+
            f"The text provides no information on what {accomplice} believes about {observer}'s belief regarding that. Therefore, it can be stated that {accomplice} is unaware of that {target} is unaware of {real_reason}.",
            # --
            f"The text provides no information on what {accomplice} believes about {observer}'s belief regarding that. Therefore, it can be stated that {accomplice} is unaware of that {target} believs that {real_reason}."
        ],
        "real_reason_accessibility": "accessible"
    })
    
    # ########### {observer} #########
    # # DA
    # # observer:liar
    # belief_qas.append({
    #     "question": f"What does {observer} believe about {liar}'s belief regarding '{real_reason_question}'?",
    #     "question_type": "tom:belief:inaccessible:reason",
    #     "tom_type": "second-order",
    #     "correct_answer": f"{observer} is unaware of {liar}'s regarding that.",
    #     "wrong_answer": f"{observer} believes that {liar} is aware of that.",
    #     "real_reason_accessibility": "inaccessible"
    # })
    # # DB
    # # observer:target
    # belief_qas.append({
    #     "question": f"What does {observer} believe about {target}'s belief regarding '{real_reason_question}'?",
    #     "question_type": "tom:belief:inaccessible:reason",
    #     "tom_type": "second-order",
    #     "correct_answer": f"{observer} is unaware of {target}'s regarding that.",
    #     "wrong_answer": f"{observer} believes that {target} is aware of that.",
    #     "real_reason_accessibility": "inaccessible"
    # })

    # # DC
    # # # observer:accomplice
    # belief_qas.append({
    #     "question": f"What does {observer} believe about {accomplice}'s belief regarding '{real_reason_question}'?",
    #     "question_type": "tom:belief:inaccessible:reason",
    #     "tom_type": "second-order",
    #     "correct_answer": f"{observer} is unaware of {accomplice}'s regarding that.",
    #     "wrong_answer": f"{observer} believes that {accomplice} is aware of that.",
    #     "real_reason_accessibility": "inaccessible"
    # })
    #######################################

    #######################################
    # **第二阶信念（关于 truth）**
    # **如果 truthQA 不是空的，加入关于 truth_question 的二阶信念**
    if truth_id == 1 and truth_question:
        ########### {liar} ########
        # AB
        # liar:target
        belief_qas.append({
            "question": f"What does {liar} believe about {target}'s belief regarding '{truth_question}'?",
            "question_type": "tom:belief:accessible:truth",
            "tom_type": "second-order",
            # ++
            "correct_answer": f"{liar} believes that {target} is unaware of that {truth}, because {target} was not involved in the conversation when this was discussed.",
            "wrong_answers": [
                # +-
                f"{liar} believes that {target} believs that {truth}.",
                # -+
                f"The text provides no information on what {liar} believes about {target}'s belief regarding that. Therefore, it can be stated that {liar} is unaware of {target} is unaware of {truth}.",
                # --
                f"The text provides no information on what {liar} believes about {target}'s belief regarding that. Therefore, it can be stated that {liar} is unaware of {target} believs that {truth}."
            ],
            "truth_accessibility": "accessible"
        })
        # AC
        # liar:accomplice
        belief_qas.append({
            "question": f"What does {liar} believe about {accomplice}'s belief regarding '{truth_question}'?",
            "question_type": "tom:belief:accessible:truth",
            "tom_type": "second-order",
            # ++
            "correct_answer": f"{liar} believes that {accomplice} believes that {truth}.",
            "wrong_answers": [
                # +-
                f"{liar} believes that {accomplice} is unaware of {truth}, because {accomplice} was not involved in the conversation when this was discussed.",
                # -+
                f"The text provides no information on what {liar} believes about {accomplice}'s belief regarding that. Therefore, it can be stated that {liar} is unaware of that {accomplice} is unaware of {truth}.",
                # --
                f"The text provides no information on what {liar} believes about {accomplice}'s belief regarding that. Therefore, it can be stated that {liar} is unaware of that {accomplice} believs that {truth}."
            ],
            "truth_accessibility": "accessible"
        })
        # AD
        # liar:observer
        belief_qas.append({
            "question": f"What does {liar} believe about {observer}'s belief regarding '{truth_question}'?",
            "question_type": "tom:belief:accessible:truth",
            "tom_type": "second-order",
            # ++
            "correct_answer":f"{liar} believes that {observer} believs that {truth}.",
            "wrong_answers": [
                # +-
                f"{liar} believes that {observer} is unaware of that {truth}, because {observer} was not involved in the conversation when this was discussed.",
                # -+
                f"The text provides no information on what {liar} believes about {observer}'s belief regarding that. Therefore, it can be stated that {liar} is unaware of that {target} believs that {truth}.",
                # --
                f"The text provides no information on what {liar} believes about {observer}'s belief regarding that. Therefore, it can be stated that {liar} is unaware of that {target} is unaware of {truth}."
            ], 
            "wrong_answers": f"{liar} is unaware of {observer}'s belief regarding that.",
            "truth_accessibility": "accessible"
        })
        ########### {target} ###########
        # BA
        # target:liar
        belief_qas.append({
            "question": f"What does {target} believe about {liar}'s belief regarding '{truth_question}'?",
            "question_type": "tom:belief:inaccessible:truth",
            "tom_type": "second-order",
            # ++
            "correct_answer": f"The text provides no information on what {target} believes about {liar}'s belief regarding that. Therefore, it can be stated that {target} is unaware of that {liar} believes that {truth}.",
            "wrong_answers": [
                # +-
                f"The text provides no information on what {target} believes about {liar}'s belief regarding that. Therefore, it can be stated that {target} is unaware of that {liar} is unaware of {truth}.",
                # -+
                f"{target} believes that {liar} believs that {truth}.",
                # --
                f"{target} believes that {liar} is unaware of that {truth}, because {liar} was not involved in the conversation when this was discussed."
            ],
            "truth_accessibility": "inaccessible"
        })
        # BC
        # target:accomplice
        belief_qas.append({
            "question": f"What does {target} believe about {accomplice}'s belief regarding '{truth_question}'?",
            "question_type": "tom:belief:inaccessible:truth",
            "tom_type": "second-order",
            # ++
            "correct_answer": f"The text provides no information on what {target} believes about {accomplice}'s belief regarding that. Therefore, it can be stated that {target} is unaware of that {accomplice} believes that {truth}.",
            "wrong_answers": [
                # +-
                f"The text provides no information on what {target} believes about {accomplice}'s belief regarding that. Therefore, it can be stated that {target} is unaware of that {accomplice} is unaware of {truth}.",
                # -+
                f"{target} believes that {accomplice} believs that {truth}.",
                # --
                f"{target} believes that {accomplice} is unaware of that {truth}, because {accomplice} was not involved in the conversation when this was discussed."
            ],
            "truth_accessibility": "inaccessible"
        })

        # BD
        # target:observer
        belief_qas.append({
            "question": f"What does {target} believe about {observer}'s belief regarding '{truth_question}'?",
            "question_type": "tom:belief:inaccessible:truth",
            "tom_type": "second-order",
            # ++
            "correct_answer": f"The text provides no information on what {target} believes about {observer}'s belief regarding that. Therefore, it can be stated that {target} is unaware of that {observer} believes that {truth}.",
            "wrong_answers": [
                # +-
                f"The text provides no information on what {target} believes about {observer}'s belief regarding that. Therefore, it can be stated that {target} is unaware of that {observer} is unaware of {truth}.",
                # -+
                f"{target} believes that {observer} believs that {truth}.",
                # --
                f"{target} believes that {observer} is unaware of that {truth}, because {observer} was not involved in the conversation when this was discussed."

            ],
            # "correct_answer": f"{target} is unaware of {observer}'s belief regarding that.",
            # "wrong_answer": f"{target} believes that {observer} is aware of that.",
            "truth_accessibility": "inaccessible"
        })

        ########### {accomplice} ########
        # CA
        # accomplice:liar
        belief_qas.append({
            "question": f"What does {accomplice} believe about {liar}'s belief regarding '{truth_question}'?",
            "question_type": "tom:belief:accessible:truth",
            "tom_type": "second-order",
            # ++
            "correct_answer": f"{accomplice} believes that {liar} believes that {truth}.",
            "wrong_answers": [
                # +-
                f"{accomplice} believes that {liar} is unaware of {truth}, because {liar} was not involved in the conversation when this was discussed.",
                # -+
                f"The text provides no information on what {accomplice} believes about {liar}'s belief regarding that. Therefore, it can be stated that {accomplice} is unaware of that {liar} is unaware of {truth}.",
                # --
                f"The text provides no information on what {accomplice} believes about {liar}'s belief regarding that. Therefore, it can be stated that {accomplice} is unaware of that {liar} believs that {truth}."
            ],
            "truth_accessibility": "accessible"
        })
        # accomplice:target
        # CB
        belief_qas.append({
            "question": f"What does {accomplice} believe about {target}'s belief regarding '{truth_question}'?",
            "question_type": "tom:belief:accessible:truth",
            "tom_type": "second-order",
            # ++
            "correct_answer": f"{accomplice} believes that {target} is unaware of that {truth}, because {target} was not involved in the conversation when this was discussed.",
            "wrong_answers": [
                # +-
                f"{accomplice} believes that {target} believs that {truth}.",
                # -+
                f"The text provides no information on what {accomplice} believes about {target}'s belief regarding that. Therefore, it can be stated that {accomplice} is unaware of {target} is unaware of {truth}.",
                # --
                f"The text provides no information on what {accomplice} believes about {target}'s belief regarding that. Therefore, it can be stated that {accomplice} is unaware of {target} believs that {truth}."
            ],
            "truth_accessibility": "accessible"
        })
        # accomplice:observer
        # CD
        belief_qas.append({
            "question": f"What does {accomplice} believe about {observer}'s belief regarding '{truth_question}'?",
            "question_type": "tom:belief:accessible:truth",
            "tom_type": "second-order",
            "correct_answer":f"{accomplice} believes that {observer} believs that {truth}.",
            "wrong_answers": [
                # +-
                f"{accomplice} believes that {observer} is unaware of that {truth}, because {observer} was not involved in the conversation when this was discussed.",
                # -+
                f"The text provides no information on what {accomplice} believes about {observer}'s belief regarding that. Therefore, it can be stated that {accomplice} is unaware of that {target} believs that {truth}."
                # --
                f"The text provides no information on what {accomplice} believes about {observer}'s belief regarding that. Therefore, it can be stated that {accomplice} is unaware of that {target} is unaware of {truth}.",
            ],
            "truth_accessibility": "accessible"
        })
        # ########### {observer} #########
        # # DA
        # # observer:liar
        # belief_qas.append({
        #     "question": f"What does {observer} believe about {liar}'s belief regarding '{truth_question}'?",
        #     "question_type": "tom:belief:accessible:truth",
        #     "tom_type": "second-order",
        #     "correct_answer": f"{observer} believes that {liar} is aware of that.",
        #     "wrong_answer": f"{observer} is unaware of {liar}'s belief regarding that.",
        #     "truth_accessibility": "accessible"
        # })
        # # DB
        # # observer:target
        # belief_qas.append({
        #     "question": f"What does {observer} believe about {target}'s belief regarding '{truth_question}'?",
        #     "question_type": "tom:belief:accessible:truth",
        #     "tom_type": "second-order",
        #     "correct_answer": f"{observer} believes that {target} is unaware of that.",
        #     "wrong_answer": f"{observer} is unaware of {target}'s belief regarding that.",
        #     "truth_accessibility": "accessible"
        # })
        # # DC
        # # # observer:accomplice
        # belief_qas.append({
        #     "question": f"What does {observer} believe about {accomplice}'s belief regarding '{truth_question}'?",
        #     "question_type": "tom:belief:accessible:truth",
        #     "tom_type": "second-order",
        #     "correct_answer": f"{observer} believes that {accomplice} is aware of that.",
        #     "wrong_answer": f"{observer} is unaware of {accomplice}'s belief regarding that.",
        #     "truth_accessibility": "accessible"
        # })
    #######################################

    return {"beliefQAs": belief_qas}

In [81]:
print(json.dumps(generate_1stbeliefQAs(data[0]), indent=4))

{
    "beliefQAs": [
        {
            "question": "What does Jamey believe about 'Who discussed 'Jamey does not want to dine out at expensive restaurants, preferring more casual dining options instead'?'?",
            "question_type": "tom:belief:accessible:reason",
            "tom_type": "first-order",
            "correct_answer": "Jamey believes that Marithza discussed Jamey does not want to dine out at expensive restaurants, preferring more casual dining options instead.",
            "wrong_answers": [
                "Jamey believes that Pearl discussed Jamey does not want to dine out at expensive restaurants, preferring more casual dining options instead.",
                "The text did not provide enough information about this, so we can state that Jamey is unaware of Marithza discussed Jamey does not want to dine out at expensive restaurants, preferring more casual dining options instead.",
                "The text did not provide enough information about this, so we c

### 4. info_QAs

In [82]:
def generate_infoAccessibilityQAs(data):
    """
    生成 infoAccessibilityQA_list 和 infoAccessibilityQAs_binary
    """
    # 获取角色
    liar = data["characters"]["liar"]
    target = data["characters"]["target"]
    accomplice = data["characters"]["accomplice"]
    observer = data["characters"]["observer"]
    
    real_reason = data["lie"]["real_reason_c"].rstrip(".")
    # truth = data["lie"]["truth_c"].rstrip(".")
    truth = (data["lie"].get("truth_c") or "").rstrip(".")
    truth_id = data["truth_id"]

    all_characters = [liar, accomplice, target, observer]
    correct_list_reason = [liar, accomplice]
    wrong_list_reason = [target, observer]

    # "information": f"{real_reason}? {liar} and {accomplice} discussed that.",
    # **生成 infoAccessibilityQA_list**
    # 之后model test怎么提供问题的？inofrmation:{inofrmation} + question:{question}
    info_access_list = [
        {
            "information": f"{real_reason}",
            "question": "List all the characters who know this information.",
            "question_type": "tom:info_accessibility:list:real_reason",
            "correct_answer": correct_list_reason,  # 正确答案是知道信息的角色
            "wrong_answers": wrong_list_reason,  # 错误答案是不知道信息的角色
            "real_reason_accessibility": "inaccessible"
        }
    ]
    

    # **如果 truth_id == 1，添加关于 truth 的信息可访问性问题**
    if truth_id == 1:
        correct_list_truth = [liar, accomplice, observer]
        wrong_list_truth = [target]

        info_access_list.append({
            "information": f"{truth}",
            "question": "List all the characters who know this information.",
            "question_type": "tom:info_accessibility:list:truth",
            "correct_answer": correct_list_truth,  # 正确答案是知道信息的角色
            "wrong_answers": wrong_list_truth,  # 错误答案是不知道信息的角色
            "missed_info_accessibility": "inaccessible"
        })
    

    # **生成 infoAccessibilityQAs_binary**
    info_access_binary = []
    
    # **关于 real_reason 的 binary question**
    for character in all_characters:
        correct_answer = "yes" if character in correct_list_reason else "no"

        info_access_binary.append({
            "information": f"{real_reason}",
            "question": f"Does {character} know this information?",
            "question_type": "tom:info_accessibility:binary:real_reason",
            "correct_answer": correct_answer,
            "missed_info_accessibility": "inaccessible"
        })
    
    # **如果 truth_id == 1，添加关于 truth 的 binary question**
    if truth_id == 1:
        for character in all_characters:
            correct_answer = "yes" if character in correct_list_truth else "no"

            info_access_binary.append({
                "information": f"{truth}",
                "question": f"Does {character} know this information?",
                "question_type": "tom:info_accessibility:binary:truth",
                "correct_answer": correct_answer,
                "missed_info_accessibility": "inaccessible"
            })

    return {
        "infoAccessibilityQA_list": info_access_list,  # 直接返回列表
        "infoAccessibilityQAs_binary": info_access_binary
    }

In [83]:
print(json.dumps(generate_infoAccessibilityQAs(data[0]), indent=4))

{
    "infoAccessibilityQA_list": [
        {
            "information": "Jamey does not want to dine out at expensive restaurants, preferring more casual dining options instead",
            "question": "List all the characters who know this information.",
            "question_type": "tom:info_accessibility:list:real_reason",
            "correct_answer": [
                "Jamey",
                "Marithza"
            ],
            "wrong_answer": [
                "Pearl",
                "Donyae"
            ],
            "real_reason_accessibility": "inaccessible"
        }
    ],
    "infoAccessibilityQAs_binary": [
        {
            "information": "Jamey does not want to dine out at expensive restaurants, preferring more casual dining options instead",
            "question": "Does Jamey know this information?",
            "question_type": "tom:info_accessibility:binary:real_reason",
            "correct_answer": "yes",
            "missed_info_accessibility": "inaccess

### 5. answerability_QAs

In [84]:
def generate_answerabilityQAs(data):
    """
    生成 answerabilityQA_list 和 answerabilityQAs_binary
    """
    # 获取角色
    liar = data["characters"]["liar"]
    target = data["characters"]["target"]
    accomplice = data["characters"]["accomplice"]
    observer = data["characters"]["observer"]

    real_reason = data["lie"]["real_reason_c"].rstrip(".")
    # truth = data["lie"]["truth_c"].rstrip(".")
    truth = (data["lie"].get("truth_c") or "").rstrip(".")
    truth_id = data["truth_id"]

    all_characters = [liar, target, accomplice]
    correct_list_reason = [liar, accomplice]  # 知道 real_reason 的角色
    correct_list_truth = [liar, accomplice, observer] if truth_id == 1 else []  # 知道 truth 的角色
    wrong_list_reason = [target, observer]
    wrong_list_truth = [target]

    # **生成 answerabilityQA_list**
    answerability_list = [
        {
            "fact_question_real_reason": f"Who discussed '{real_reason}'?",
            "question": "List all the characters who know the precise correct answer to this question.",
            "question_type": "tom:answerability:list:real_reason",
            "correct_answer": correct_list_reason,  # 知道 real_reason 的角色
            "wrong_answers": wrong_list_reason,  # 不知道 real_reason 的角色
            "real_reason_accessibility": "inaccessible"
        }
    ]

    # **如果 truth_id == 1，添加关于 truth 的 answerability list question**
    if truth_id == 1:
        answerability_list.append({
            "fact_question_truth": f"Who discussed '{truth}'?",
            "question": "List all the characters who know the precise correct answer to this question.",
            "question_type": "tom:answerability:list:truth",
            "correct_answer": correct_list_truth,  # 知道 truth 的角色
            "wrong_answers": wrong_list_truth,  # 不知道 truth 的角色
            "truth_accessibility": "inaccessible"
        })

    # **生成 answerabilityQAs_binary**
    answerability_binary = []
    
    # **关于 real_reason 的 binary question**
    for character in all_characters:
        correct_answer = "yes" if character in correct_list_reason else "no"

        answerability_binary.append({
            "fact_question_real_reason": f"Who discussed '{real_reason}'?",
            "question": f"Does {character} know the precise correct answer to this question?",
            "question_type": "tom:answerability:binary:real_reason",
            "correct_answer": correct_answer,
            # "real_reason_accessibility": "inaccessible"
        })
    
    # **如果 truth_id == 1，添加关于 truth 的 binary question**
    if truth_id == 1:
        for character in all_characters:
            correct_answer = "yes" if character in correct_list_truth else "no"

            answerability_binary.append({
                "fact_question_truth": f"Who discussed '{truth}'?",
                "question": f"Does {character} know the precise correct answer to this question?",
                "question_type": "tom:answerability:binary:truth",
                "correct_answer": correct_answer,
                # "truth_accessibility": "inaccessible"
            })

    return {
        "answerabilityQA_list": answerability_list,  # 直接返回列表
        "answerabilityQAs_binary": answerability_binary
    }

In [85]:
print(json.dumps(generate_answerabilityQAs(data[0]), indent=4))

{
    "answerabilityQA_list": [
        {
            "fact_question_real_reason": "Who discussed 'Jamey does not want to dine out at expensive restaurants, preferring more casual dining options instead'?",
            "question": "List all the characters who know the precise correct answer to this question.",
            "question_type": "tom:answerability:list:real_reason",
            "correct_answer": [
                "Jamey",
                "Marithza"
            ],
            "wrong_answer": [
                "Pearl",
                "Donyae"
            ],
            "real_reason_accessibility": "inaccessible"
        }
    ],
    "answerabilityQAs_binary": [
        {
            "fact_question_real_reason": "Who discussed 'Jamey does not want to dine out at expensive restaurants, preferring more casual dining options instead'?",
            "question": "Does Jamey know the precise correct answer to this question?",
            "question_type": "tom:answerability:binary:rea

### 6. liedetectability

In [87]:
def generate_liedetectabilityQAs(data):
    """
    生成 liedetectabilityQAs_list 和 liedetectabilityQAs_binary
    """
    # 获取角色
    liar = data["characters"]["liar"]
    target = data["characters"]["target"]
    accomplice = data["characters"]["accomplice"]
    observer = data["characters"]["observer"]

    truth_id = data["truth_id"]
    lie = data["lie"]["lie_c"].rstrip(".")
    real_reason = data["lie"]["real_reason_c"].rstrip(".")
    # truth = data["lie"]["truth_c"].rstrip(".")
    truth = (data["lie"].get("truth_c") or "").rstrip(".")
    falsification = bool(data["lie"]['falsification'])  # 确保 falsification 是布尔值

    all_characters = [liar, target, accomplice]
    correct_list_reason = [liar, accomplice]  # 知道 real_reason 的角色
    correct_list_truth = [liar, accomplice]  # 知道 truth 的角色, observer虽然知道truth但是不知道lie
    wrong_list_reason = [target]
    wrong_list_truth = [target]

    # **初始化列表**
    liedetectability_list = []

    # **生成 liedetectabilityQAs_list**
    if truth_id == 1 and falsification:  # 当 truth 存在且是篡改信息时
        liedetectability_list.append({
            "fact_question_truth": f"Was what {liar} told {target}, '{lie}', true?",
            "question": "List all the characters who know the precise correct answer to this question.",
            "question_type": "tom:liedetectability:list",
            "correct_answer": correct_list_truth,  # 包含 observer
            "wrong_answers": wrong_list_truth,  # 只有 target 不知道
        })
    else:
        liedetectability_list.append({
            "fact_question_truth": f"Was what {liar} told {target}, '{lie}', true?",
            "question": "List all the characters who know the precise correct answer to this question.",
            "question_type": "tom:liedetectability:list",
            "correct_answer": correct_list_reason,  # 只有 liar 和 accomplice 知道
            "wrong_answers": wrong_list_reason,  # observer 和 target 不知道
        })

    # **生成 liedetectabilityQAs_binary**
    liedetectability_binary = []

    if truth_id == 1 and falsification:  # 当 truth 存在且是篡改信息时
        for character in all_characters:
            correct_answer = "yes" if character in correct_list_truth else "no"

            liedetectability_binary.append({
                "fact_question_truth": f"Was what {liar} told {target}, '{lie}', true?",
                "question": f"Does {character} know the precise correct answer to this question?",
                "question_type": "tom:liedetectability:binary",
                "correct_answer": correct_answer,
            })
    else:
        for character in all_characters:
            correct_answer = "yes" if character in correct_list_reason else "no"

            liedetectability_binary.append({
                "truth_question": f"Was what {liar} told {target}, '{lie}', true?",
                "question": f"Does {character} know the precise correct answer to this question?",
                "question_type": "tom:liedetectability:binary",
                "correct_answer": correct_answer,
            })

    return {
        "liedetectabilityQAs_list": liedetectability_list,  # 直接返回列表
        "liedetectabilityQAs_binary": liedetectability_binary
    }

In [88]:
print(json.dumps(generate_liedetectabilityQAs(data[0]), indent=4))

{
    "liedetectabilityQAs_list": [
        {
            "fact_question_truth": "Was what Jamey told Pearl, 'Jamey need to work this weekend, so Jamey can\u2019t make it', true?",
            "question": "List all the characters who know the precise correct answer to this question.",
            "question_type": "tom:liedetectability:list",
            "correct_answer": [
                "Jamey",
                "Marithza"
            ],
            "wrong_answer": [
                "Pearl"
            ]
        }
    ],
    "liedetectabilityQAs_binary": [
        {
            "truth_question": "Was what Jamey told Pearl, 'Jamey need to work this weekend, so Jamey can\u2019t make it', true?",
            "question": "Does Jamey know the precise correct answer to this question?",
            "question_type": "tom:liedetectability:binary",
            "correct_answer": "yes"
        },
        {
            "truth_question": "Was what Jamey told Pearl, 'Jamey need to work this weekend,

### 7. lieability

In [94]:
def generate_lieabilityQAs(data):
    """
    生成 lieability
    """
    # 获取角色
    liar = data["characters"]["liar"]
    target = data["characters"]["target"]
    lie = data["lie"]["lie_c"].rstrip(".")
    real_reason = data["lie"]["real_reason_c"].rstrip(".")
    truth = (data["lie"].get("truth_c") or "").rstrip(".")

    lieability = []

    # First question: Explanation based on truth
    lieability.append({
        "fact_question_truth": f"Was what {liar} told {target}, '{lie}', true?",
        "question": f"Please choose the correct answer from the options below:",
        "question_type": "tom:lieability:binary",
        "correct_answer": [f"{liar} tell {target} the {lie}' because {liar} believes that {target} is unaware of {truth}."], #++
        "wrong_answers": [
            f"{liar} tell {target} the {lie}' because {liar} is unaware that {target} believes {truth}.",
            f"{liar} tell {target} the {lie}' because {liar} believes that {target} believes {truth}.",
            f"{liar} tell {target} the {lie}' because {liar} is unaware that {target} is unaware of {truth}."
        ],
    })

    # Second question: Explanation based on real_reason
    lieability.append({
        "fact_question_truth": f"Was what {liar} told {target}, '{lie}', true?",
        "question": f"Please choose the correct answer from the options below:",
        "question_type": "tom:lieability:binary",
        "correct_answer": [f"{liar} tell {target} the {lie}' because {liar} believes that {target} is unaware of {real_reason}."],
        "wrong_answers": [
            f"{liar} tell {target} the {lie}' because {liar} is unaware that {target} believes {real_reason}.",
            f"{liar} tell {target} the {lie}' because {liar} believes that {target} believes {real_reason}.",
            f"{liar} tell {target} the {lie}' because {liar} is unaware that {target} is unaware of {real_reason}."
        ],
    })

    return {
        "lieability": lieability,
    }

### 8. put into jsonl

In [90]:
def merge_beliefQAs(d1, d2):
    """
    d1 和 d2 都是形如 {"beliefQAs": [...]} 的字典
    将 d1 和 d2 的 "beliefQAs" 列表合并到同一个字典中
    """
    # 如果你想要新建一个字典，不修改原对象，可以这样：
    return {
        "beliefQAs": d1["beliefQAs"] + d2["beliefQAs"]
    }

In [95]:
def assign_question_ids(new_entry: dict, set_id: str) -> dict:
    """
    给 new_entry 中的每个问题加上 q_id。
    假设以下字段都存储为 list：
      - beliefQAs
      - infoAccessibilityQA_list
      - infoAccessibilityQAs_binary
      - answerabilityQA_list
      - answerabilityQAs_binary
      - liedetectabilityQAs_list
      - liedetectabilityQAs_binary
      - lieabilityQAs
    并且你希望每个问题都能分配到一个唯一的 q_id。

    参数:
        new_entry (dict): 包含多个 QA 列表的字典。
        set_id (str): 用作 q_id 前缀 (例如 "0-0-0")

    返回:
        dict: 在原字典基础上修改并返回，给每条问题都添加了 "q_id" 字段
    """
    # comprehensionQA
    if "comprehensionQA" in new_entry:
        for i, q_dict in enumerate(new_entry["comprehensionQA"]):
            q_dict["q_id"] = f"{set_id}-comprehension-{i}"

    # justificationQA
    if "justificationQA" in new_entry:
        for i, q_dict in enumerate(new_entry["justificationQA"]):
            q_dict["q_id"] = f"{set_id}-justification-{i}"


    # reasonQA
    if "reasonQA" in new_entry:
        for i, q_dict in enumerate(new_entry["reasonQA"]):
            q_dict["q_id"] = f"{set_id}-fact_reason-{i}"

    # truthQA
    if "truthQA" in new_entry:
        for i, q_dict in enumerate(new_entry["truthQA"]):
            q_dict["q_id"] = f"{set_id}-fact_truth-{i}"

    # beliefQAs
    if "beliefQAs" in new_entry:
        for i, q_dict in enumerate(new_entry["beliefQAs"]):
            q_dict["q_id"] = f"{set_id}-belief-{i}"

    # infoAccessibilityQA_list
    if "infoAccessibilityQA_list" in new_entry:
        for i, q_dict in enumerate(new_entry["infoAccessibilityQA_list"]):
            q_dict["q_id"] = f"{set_id}-info_access_list-{i}"

    # infoAccessibilityQAs_binary
    if "infoAccessibilityQAs_binary" in new_entry:
        for i, q_dict in enumerate(new_entry["infoAccessibilityQAs_binary"]):
            q_dict["q_id"] = f"{set_id}-info_access_binary-{i}"

    # answerabilityQA_list
    if "answerabilityQA_list" in new_entry:
        for i, q_dict in enumerate(new_entry["answerabilityQA_list"]):
            q_dict["q_id"] = f"{set_id}-answerability_list-{i}"

    # answerabilityQAs_binary
    if "answerabilityQAs_binary" in new_entry:
        for i, q_dict in enumerate(new_entry["answerabilityQAs_binary"]):
            q_dict["q_id"] = f"{set_id}-answerability_binary-{i}"

    # liedetectabilityQAs_list
    if "liedetectabilityQAs_list" in new_entry:
        for i, q_dict in enumerate(new_entry["liedetectabilityQAs_list"]):
            q_dict["q_id"] = f"{set_id}-liedetect_list-{i}"

    # liedetectabilityQAs_binary
    if "liedetectabilityQAs_binary" in new_entry:
        for i, q_dict in enumerate(new_entry["liedetectabilityQAs_binary"]):
            q_dict["q_id"] = f"{set_id}-liedetect_binary-{i}"
    
    # lieabilityQAs
    if "lieabilityQAs" in new_entry:
        for i, q_dict in enumerate(new_entry["lieabilityQAs"]):
            q_dict["q_id"] = f"{set_id}-lieability-{i}"

    return new_entry

In [92]:
with open("/Users/liuyiwei/Desktop/TactfulToM/dataset/Tactful_conv_element.json", "r", encoding="utf-8") as file:
    content = file.read().strip()
    if content == "":
        data = {}  # 或者 data = []，取决于你期望的格式
    else:
        data = json.loads(content)

In [109]:
# 你可以手动指定要处理的 set_id，例如 "0-0-0"
selected_set_id = "0-4-0-2"

# 选择要处理的 set
selected_set = None
for item in data:
    if item["set_id"] == selected_set_id:
        selected_set = item
        break
if selected_set is None:
    print(f"Set ID {selected_set_id} not found.")
else:
    print(f"Processing set: {selected_set_id}")

    
    # **生成问题** 遍历所有的问题generation函数
    comprehension_qas = generate_comprehensionQA(selected_set)
    justification_qas = generate_justificationQA(selected_set)

    fact_qas = generate_fact_QA(selected_set)

    belief_qas_1 = generate_1stbeliefQAs(selected_set)
    belief_qas_2 = generate_2ndbeliefQAs(selected_set)
    belief_qas = merge_beliefQAs(belief_qas_1, belief_qas_2)

    info_accessibility_qas = generate_infoAccessibilityQAs(selected_set)
    answerability_qas = generate_answerabilityQAs(selected_set)
    
    liedetectability_qas = generate_liedetectabilityQAs(selected_set)  # ✅ 新增的 Lie Detectability 逻辑

    lieability_qas = generate_lieabilityQAs(selected_set)  # ✅ 新增的 Lie Detectability 逻辑

    # 如果只想保留原始set中的一部分信息
    selected_fields = {key: selected_set[key] for key in ["set_id", "lie_id", "conv_id", "truth", "lie_type", "emotion"] if key in selected_set}
    
    # **合并生成的数据**
    new_entry = {
        **selected_set,  # 复制原始数据
        **comprehension_qas,
        **justification_qas,
        **fact_qas,
        **belief_qas,
        **info_accessibility_qas,
        **answerability_qas,
        **liedetectability_qas,  # ✅ 加入 Lie Detectability QAs
        **lieability_qas,  # ✅ 加入 Lie Detectability QAs
    }
    
    # 调用assign_question_ids给每条 reasonQA / truthQA 的问题添加唯一的 q_id
    new_entry = assign_question_ids(new_entry, set_id=selected_set_id)

    # **写入 `whitetom_set.json`**
    output_filename = "/Users/liuyiwei/Desktop/TactfulToM/dataset/Tactful_conv_question.json"

    # 读取 JSON 文件（如果文件不存在，则创建一个空列表）
    try:
        with open(output_filename, "r", encoding="utf-8") as file:
            existing_data = json.load(file)  # 读取已有数据
    except (FileNotFoundError, json.JSONDecodeError) as e:
        print(f"Error loading JSON (likely file not found or empty): {e}")
        existing_data = []  # 如果文件不存在，则初始化为空列表

    # **检查是否已有该 `set_id`，如果有就覆盖**
    found = False
    for i, entry in enumerate(existing_data):
        if entry["set_id"] == selected_set_id:
            existing_data[i] = new_entry  # **覆盖旧数据**
            found = True
            break

    if not found:
        existing_data.append(new_entry)  # **如果没有，才追加**

    # **写回文件**
    with open(output_filename, "w", encoding="utf-8") as file:
        json.dump(existing_data, file, indent=4, ensure_ascii=False)

    print(f"Data for set {selected_set_id} written to {output_filename}")

    # **立即检查是否写入成功**
    with open(output_filename, "r", encoding="utf-8") as file:
        content = file.read()
        print("Written JSON Content:\n", content)  # 检查是否写入了内容

Processing set: 0-4-0-2
Data for set 0-4-0-2 written to /Users/liuyiwei/Desktop/TactfulToM/dataset/Tactful_conv_question.json
Written JSON Content:
 [
    {
        "set_id": "0-1-0-0",
        "lie_id": "0-0",
        "conv_id": 1,
        "truth_id": 0,
        "lie_type": "altruistic_white_lies",
        "emotion": "sad",
        "topic": {
            "scenario": "favorite cuisine styles",
            "situation_topic": "the plan for this weekend",
            "situation": "Pearl is inviting Jamey for having dinner at a new opened fancy Italian restaurant together this weekend",
            "lie_objective": "decline the invitation",
            "leave_reason_B": "need to make a phone call",
            "leave_reason_D_1": "remembered to submit some papers",
            "leave_reason_D_2": "have a meeting starting soon that I need to prepare for"
        },
        "relationship": "coworkers",
        "characters": {
            "liar": "Jamey",
            "target": "Pearl",
      